### Generate Origin-Destination Matrix [Updated]

Create a method `generate_odmatrix` that takes in a `date_range` input parameter and returns a `pandas.DataFrame` with the `trips.csv` `pickup_loc_name` as the row names (dataframe index) and `dropoff_loc_name` as the columns. The values for each row-column combination is the average daily number of vehicle trips that occured within the `date_range` specified.

  - `date_range` : a *range saerch* that takes a tuple of datetime strings, and filters trips based on `pickup_datetime`. Defaults to `None`, in which case all dates are included.
      - Case 1: tuple like `(value,None)` returns all entries from `value` (end inclusive) sorted chronologically 
      - Case 2: tuple like `(None,value)` returns all entries up to `value` (end inclusive) sorted chronologically 
      - Case 3: tuple like `(value1,value2)`  returns values between `value1` and `value2` (end inclusive) sorted chronologically
      
Input errors to the `date_range` parameter should be handled like that of `search_trips`.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def generate_odmatrix(date_range):
    
    try:
        trips = pd.read_csv(f'trips_test2.csv')
        locations = pd.read_csv(f'locations.csv')

        trips['pickup_datetime'] = pd.to_datetime(trips['pickup_datetime'], format='%H:%M:%S,%d-%m-%Y')
        trips['count'] = 1
        
        if len(date_range) > 2:
            raise ValueError # raise SakayDBError

        if date_range is None:
            trips = trips.copy()

        elif date_range[1] is None:
            date_range = [pd.to_datetime(x, format='%H:%M:%S,%d-%m-%Y') for x in date_range]
            trips = trips[trips['pickup_datetime']>=date_range[0]]

        elif date_range[0] is None:
            date_range = [pd.to_datetime(x, format='%H:%M:%S,%d-%m-%Y') for x in date_range]
            trips = trips[trips['pickup_datetime']<=date_range[1]]

        elif date_range[0] is not None and date_range[1] is not None:
            date_range = [pd.to_datetime(x, format='%H:%M:%S,%d-%m-%Y') for x in date_range]
            trips = trips[(trips['pickup_datetime']>date_range[0]) & (trips['pickup_datetime']<=date_range[1])]

        trips = trips.merge(locations, left_on='pickup_loc_id', right_on = 'location_id', how = 'left').rename(columns={'loc_name':'pickup_loc_name'})
        trips = trips.merge(locations, left_on='dropoff_loc_id', right_on = 'location_id', how = 'left').rename(columns={'loc_name':'dropoff_loc_name'})
        trips = trips.drop(columns=['location_id_x', 'location_id_y', 'pickup_loc_id', 'dropoff_loc_id'])

        trips = trips[['count','pickup_loc_name', 'dropoff_loc_name', 'pickup_datetime']].copy()
        trips['pickup_date'] = trips['pickup_datetime'].dt.strftime('%Y-%m-%d') 
        trips.drop(columns='pickup_datetime', inplace = True)

        od_matrix = trips.groupby(['pickup_loc_name', 'dropoff_loc_name', 'pickup_date'], as_index=False).sum()
        od_matrix = od_matrix.pivot_table(columns='pickup_loc_name', index='dropoff_loc_name', aggfunc='mean', fill_value=0)
        od_matrix.columns = [x[1] for x in od_matrix.columns]

        return od_matrix
    
    except:
        raise ValueError # raise SakayDBError